<p>In this notebook, we introduce how to query the anotation database and give a flavor of how to use the results to do some simple analysis. By the end, you should be able to query for neurons, query for synapses between those neurons, and visualize the neurons and synapes.

</div>



<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<h2>Background</h2>


This dataset is part of a collaboration between the Lee Lab at Harvard University and the Tuthill Lab at the University of Washington. Before beginning on this tutorial, please make sure you have completed the onboarding instructions which can be found [here](https://docs.google.com/document/d/10fnQHWTPluKeNXko6C63iKTKVwqXeSEQ10ftYgtEfhc/edit)

</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<h2>Local setup instructions</h2>

<p>
To query and visualize the data, you will need a few custom packages developed by the Allen Institute and Princeton University.  If you want to setup your local computer's python system to use these packages you'll need to install them and their dependancies. 
</p>

<p>
The easiest way to install is to use Anaconda python.  Open up a terminal, or a command prompt on windows.  Navigate to the directory where you have checked out the swdb_2021 repository. Activate a conda environment if you'd like then type..

</div>
<h4>Linux/OSX</h4>
<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #DFF0D8; ">
./scripts/em_conda_install.sh
</div>
<h4>Windows</h4>
<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #DFF0D8; ">
./scripts/em_conda_install.bat
</div>
<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
You'll need to restart your kernel after you do the install
</div>




<h2>3D Visualization Methods </h2>

EM data involves high resolution reconstructions of neurons, and so visualizing them interatively in 3D is essential to understanding the data (plus it's just fun).  This requires use of more specialized plotting packages than matplotlib.  There are a few options that have different plusses and minuses that are summarized below.  We'll show you code snippets from all methods, but execute the one based upon the viz_method variable defined in the next cell.
<table>
    <tr>
        <td><h3>viz_method</h3></td>
        <td><h3>pros</h3></td>
        <td><h3>cons</h3></td>
    </tr>
    <tr>
        <td>
           vtk
        </td>
        <td>Fastest <br/>
            Most features: <br/>
            mesh coloring<br/>  
            programatic camera control<br/>
            programatic saving<br/>
            extensible</td>
        <td> Requires local installation <br/>(no AWS possible)</td>
    </tr>
    <tr>
        <td> itkwidgets</td>
        <td> fastest notebook widget <br/>
             easy screen shot <br/>
             bug free <br/>
             aws compatible <br/>
        </td>
        <td>
        no programatic camera control<br/>
        no advanced mesh coloring
        </td> 
    </tr>    
    </table>

In [ ]:
import os

# viz_method = one of ['itkwidgets', 'vtk']
viz_method = 'vtk'


<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<h2>Import the key modules</h2>
Our analysis workflows use a couple of special purpose packages we have developed. Here, we are going to look at the CAVEclient (Connectome Annotation Versioning Engine), a package that quickly performs simple analysis queries and produces tidy Pandas dataframes.
</div>

In [ ]:
# this is the EM specific package for querying the EM data
from caveclient import CAVEclient


<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<h2>Import some of our favorite modules</h2>
Our analysis workflows makes use of many of the common scientific computing packages like Numpy and Pandas.
</div>

In [ ]:
# import some of our favorite packages
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline


<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<h2>Setting up a CAVE account </h2>

CAVE is the Connectome Annotation Versioning Engine.
It is a collection of services which manage proofreading and data annotation for large scale connectomics dataset.
These services were developed during the IARPA MICrONS program, but now support data from several projects including FANC and FlyWire.  You need to have an account on CAVE to use its services.


To get an account please first go to this [website](https://cave.fanc-fly.com/materialize/views/datastack/fanc_production_mar2021)

You should be prompted to login with a google account and then you must seperately accept the terms and conditions.

Once you have registered, you need to get access to the dataset. If you have not already, please complete the [onboarding instructions](https://docs.google.com/document/d/10fnQHWTPluKeNXko6C63iKTKVwqXeSEQ10ftYgtEfhc/edit) of for using FANC. Once you have done so, please reach out to one of the following people with your email address to get access to the dataset:
   
* Jasper Phelps (jtmaniatesselvin@g.harvard.edu)
* Leila Elabbady (elabbady@uw.edu)
* John Tuthill (tuthill@uw.edu)
* Wei-Chung Lee (wei-chung_lee@hms.harvard.edu)
 
Once you have an account and have access to the FANC dataset, you can then follow along this notebook.
This notebook will show you how to get a programatic token that you can use to authenticate to the services in order to make queries. The token can then be used on different computers.

<h2>Setting key parameters </h2>
Every combination of image and segmentation data we call a 'datastack', since a dataset might have multiple such 'datastacks' associated with it.  The female ventral nerve cord that we are going to use is called `fanc_v4` named because....
</div>


In [ ]:
client = CAVEclient()

if not os.path.isfile(os.path.expanduser("~/.cloudvolume/secrets/cave-secret.json")):
    client.auth.get_new_token(open=True)

In [ ]:
# if you have not yet setup this computer, uncomment this below line
# paste the token from the website in, and run the line

# client.auth.save_token(token="paste_token_here", overwrite=True)

# then comment or delete the line as you don't need to run it on this computer  again

In [ ]:
datastack_name = 'fanc_production_mar2021'

client = CAVEclient(datastack_name)

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<h2>Terms of Service Error</h2>
If for some reason you are getting an error that says you haven't signed the terms of service, visit this link.

<a href="https://global.daf-apis.com/sticky_auth/api/v1/tos/2/accept"> Microns Public TOS </a>
</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<h2>Client Basics</h2>
The client lets you connect to a number of different services, and you can read about all of them in  <a href="https://caveclient.readthedocs.io/">the CAVEclient documentation</a>

For this notebook however we will focus on the 'materialize' client which lets you access tables of annotations on the data, including cells, nuclei and synapses. 

To see what tables are available, use the 'get_tables' method.
</div>

In [ ]:
client.materialize.get_tables()

In [ ]:
# For each table you can get metadata about that table
client.materialize.get_table_metadata('soma_jan2022')

In [ ]:
# and how many annotations are in it
# the maybe most important table is the synapse table (synapses_pni_2)
# which has >330 million synapses, so querying its length takes a long time
client.materialize.get_annotation_count('soma_jan2022')


In [ ]:
# lets look at the soma table
# if you ask for the whole thing, the client will give you the first 200K rows
# which is good enough to cover everything other than synapses
soma_df= client.materialize.query_table('soma_jan2022')
soma_df.head()

In [ ]:
# remember that you can check the metadata if you're unsure what is in the table
client.materialize.get_table_metadata('soma_jan2022')['description']

In [ ]:
# useful function for converting a pandas data frame voxel position
# column to a np.array of Nx3 size in nm
def convert_to_nm(col, voxel_size=[4.3,4.3,45]):
    return np.vstack(col.values)*voxel_size

# function to plot a dataframe
def plot_soma_loc(df, ax, x=0, y=1, c='r', s=10, cmap=None):
    
    # convert the position to microns
    pos = convert_to_nm(df['pt_position'])/1000
    
    if cmap:
        c = pos[:,y]
        ax.scatter(pos[:,x], pos[:,y], c=c, s=s)
    # plot two dimensions as a scatterplot
    else:
        ax.scatter(pos[:,x], pos[:,y], c=c, s=s)

# make a new axis with two subplots
fig = plt.figure(figsize=(10,6))
ax1 = fig.add_subplot(1, 2, 1)
ax2 = fig.add_subplot(1, 2, 2, sharex=ax1)

# plot each soma
plot_soma_loc(soma_df, ax1, cmap='viridis', s=1)
#plot_soma_loc(ext_axon_df, ax1, c='r')


# label some axis, make x,y scaling same
ax1.set_xlabel('x (um)')
ax1.set_ylabel('y (um)')
# to keep the pia 'up'
#ax1.set_ylim(1100,300)
ax1.set_aspect('equal')

# plot the same thing as x,z
plot_soma_loc(soma_df, ax2, x=2, y=1, cmap='viridis', s=1)
#plot_soma_loc(ext_axon_df, ax2, x=0, y=2, c='r')

ax2.set_xlabel('z (um)')
ax2.set_ylabel('y (um)')
ax2.set_aspect('equal')

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<h2>Querying for synapses</h2>
Let's pick out a cell ID of a tibia flexor motor neuron and then find all the synapses onto that neuron</div>


In [ ]:
motor_neuron = 648518346503918688
motor_neuron

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

Now we are ready to query synapses for this neuron. Because the analysis database can, in principle, handle multiple different sources of synapses, we have to specify which synapse table we want to use. The current table holding the most up to date automated synapse detection is called `synapses_may2021`.  The "info" service has this information however, so we don't need to remember it.

Because synapses are such a common query, there is a wrapper method `synapse_query` which makes uses the default synapse table and simplifies the syntax. So, we can specify which synapses we get back from the table by setting the `post_ids` argument to a list of IDs, which makes a query that only returns synapses whose postsynaptic id is the one selected.  You can also filter by `pre_ids` which will do the same for the pre-synaptic side. Setting this will only return synapses that are from certain neurons.  Setting both `pre_ids` and `post_ids` will return only synapses that are from the `pre_ids` onto the `post_ids`.
</div>

In [ ]:
print(client.materialize.synapse_table)
post_synapse_df = client.materialize.synapse_query(post_ids = motor_neuron)
# lets post the shape to see how many synapses we have
print(post_synapse_df.shape)
# and take a peak at the whole dataframe
post_synapse_df.head()

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

<h3> Synapse Table Descriptions</h3>
Here's a breakdown of what each of those columns mean.
</div>

<table style={float:left}>
    <tr>
        <td><h4>column</h4></td>
        <td><h4>description</h4></td>
    </tr> 
    <tr>
        <td>id</td>
        <td>The ID that is specific to this synapse annotation</td>
    </tr>
    <tr>
        <td>pre_pt_supervoxel_id</td>
        <td>a bookkeeping column for the presynaptic side </td>
    </tr>
    <tr>
        <td>pre_pt_root_id</td>
        <td>the ID of the cell on the presynaptic side</td>
    </tr>
    <tr>
        <td>post_pt_supervoxel_id</td>
        <td>Same bookkeeping column as pre_pt but for the post synaptic side.</td>
    </tr>
    <tr>
        <td>post_pt_root_id</td>
        <td>Same as pre_pt but for the post synaptic side</td>
    </tr>
    <tr>
        <td>score</td>
        <td>The size of the synaptic cleft in units of 4.3,4.3,45 voxels.</td>
    </tr>
    <tr>
        <td>pre_pt_position</td>
        <td>a point that is in the pre-synaptic terminal of this synapse (in voxels) </td>
    </tr>
    <tr>
        <td>post_pt_position</td>
        <td>Same as pre_pt but for the post synaptic side.</td>
    </tr>

</table>
<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

Each row in this table is a single synapse. You should see that the value in the post_pt_root_id column is the same for all, and equals the id you selected above. 
<h5>Note again that position columns are in voxel coordinates, just like Neuroglancer displays in the upper left corner. A single voxel has dimensions 4.3x4.3x45 nm.</h5>
</div>

In [ ]:
# Lets make a matrix of synapse positions for this neuron
syn_pos_nm = convert_to_nm(post_synapse_df.post_pt_position)
# and plot their positions in x,y along with the soma position as a red circle
f ,ax =plt.subplots(1,2,figsize=(12,6))
ax[0].scatter(syn_pos_nm[:,0]/1000, syn_pos_nm[:,1]/1000, alpha=0.6, s=5, c=post_synapse_df.score, cmap='terrain')

# add the soma position as a red circle
soma_pos = convert_to_nm(soma_df.pt_position[soma_df.pt_root_id == motor_neuron])
ax[0].scatter([soma_pos[0,0]/1000],[soma_pos[0,1]/1000],c='r',s=150)
ax[0].set_aspect('equal')
ax[0].set_xlabel('x (um)')
ax[0].set_ylabel('y (um)')


m =ax[1].scatter(syn_pos_nm[:,0]/1000, syn_pos_nm[:,2]/1000, alpha=0.6, s=5, c=post_synapse_df.score, cmap='terrain')

# add the soma position as a red circle
soma_pos = convert_to_nm(soma_df.pt_position[soma_df.pt_root_id == motor_neuron])
ax[1].scatter([soma_pos[0,0]/1000],[soma_pos[0,2]/1000],c='r',s=150)
ax[1].set_aspect('equal')
ax[1].set_xlabel('x (um)')
ax[1].set_ylabel('z (um)')
f.colorbar(m, label='Synapse Size')

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

Now lets figure out which premotor neuron makes the most synapses onto this tibia flexor
</div>

In [ ]:
# pandas groupby to count number of synapses from different neurons
# followed by transform to label the individual synapses with 
# how many other synapses are in that connection.

# transform labels all the rows in the group with the result of this function on the group
syn_in_conn=post_synapse_df.groupby('pre_pt_root_id').transform(len)['id']
# save this result in a new colum
post_synapse_df['syn_in_conn']=syn_in_conn
post_synapse_df[['id', 'pre_pt_root_id', 'score','syn_in_conn']].sort_values('syn_in_conn', ascending=False).head()

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

Let's go ahead and get the ID of that premotor neuron which will have the largest number of synapses onto this tibia flexor
</div>

In [ ]:
# find the neuron with the most synapses
max_input_idx = post_synapse_df.syn_in_conn.idxmax()
max_input_neuron = post_synapse_df.loc[max_input_idx].pre_pt_root_id
print(max_input_neuron)

# the shape of this synapse table will tell us how many synapses are made by this prem
post_synapse_df[post_synapse_df.pre_pt_root_id == max_input_neuron].shape

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<h2>Linking synapses to somas</h2>
You might want to link synapses to the cell bodies or cell classes of the pre or post-synaptic cells.  To do this, you want to 'merge' (See <a href="https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html"> pandas merge docs </a> ) the information between a synapse table and a table of somas. </div>


In [ ]:
# merging in the neuron table using 'left' ensures that every row
# in the first dataframe passed (the synapse table) 
# has at least one entry in the result
# more than 1 if there is more than one on the 'right'
# (the second table passed)
synapse_and_soma_df = pd.merge(post_synapse_df, soma_df,
         left_on='post_pt_root_id',
         right_on='pt_root_id',
         how='left',
         suffixes=['_syn', '_soma'])

# now pt_position is the soma position
# id_syn is the ID of the synapse
# id_soma is the ID of the soma that had the matched root_id
# pt_position is the position of that soma
synapse_and_soma_df[['id_syn',
                     'pre_pt_root_id',
                     'post_pt_root_id',
                     'id_soma',
                     'pt_position']]

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
    <h3>Mesh visualization</h3>
Now we can plot these synapses in 2d, but we have the detailed 3d morphology of these neurons, so why don't we look at them!
 </div>

In [ ]:
# this is an EM specific package for getting meshes
# and doing analysis on those meshes
from meshparty import trimesh_io, trimesh_vtk
from meshparty import skeletonize, skeleton_io, skeleton
import cloudvolume

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
To access the 3d meshes of neurons, we need use a MeshMeta object, make sure to pass the segmentation source (which you can get from the client!) and the segmentation id that you are interested in. In our case, we want to see the motor neuron and the strongest premotor neuron
</div>

In [ ]:
from pathlib import Path
import json
        
seg_source = client.info.segmentation_source()

mm = trimesh_io.MeshMeta(cv_path =seg_source,
                         disk_cache_path='../meshes',map_gs_to_https=True)
motor_neuron_mesh = mm.mesh(seg_id = motor_neuron, remove_duplicate_vertices=True, merge_large_components=True)

In [ ]:
# load the meshes
premotor_mesh = mm.mesh(seg_id = max_input_neuron,remove_duplicate_vertices=True, merge_large_components=True)

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
    
Meshes are triangular meshes, that are composed of vertices (N,3) and faces (N,3) which are indices into the vertex list
    

The Mesh class is based upon trimesh (<a href="https://github.com/mikedh/trimesh"> https://github.com/mikedh/trimesh</a>), with some added features for doing graphs

You can find the source code for mesh party here .. <a href="https://github.com/sdorkenw/MeshParty">https://github.com/sdorkenw/MeshParty </a>
    </div>

In [ ]:
motor_neuron_mesh.vertices.shape, motor_neuron_mesh.faces.shape

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

Below we are going to visualize things in 3d.  Different code paths  illustrate different visualization methods
</div>


In [ ]:
# Set viewer equals None so vtk option doesn't error
viewer = None
if viz_method == 'itkwidgets':
    #import ITK widgets view function
    from itkwidgets import view
    
    # step 1
    # convert your actors to vtkpolydata objects
    post_poly_data = trimesh_vtk.trimesh_to_vtk(motor_neuron_mesh.vertices, motor_neuron_mesh.faces, None)
    pre_poly_data = trimesh_vtk.trimesh_to_vtk(premotor_mesh.vertices, premotor_mesh.faces, None)

    # step 2
    # then create a viewer with this view function
    # pass in polydata objects, what colors you want
    # see docstring for more options
    prepostviewer=view(geometries=[post_poly_data, pre_poly_data],
                ui_collapsed=True)

    # viewer controls..
    # pinch movements:  to zoom in and out
    # ctrl+wheel: on a mouse to do the same
    # shift+drag: to pan

    display(prepostviewer)
elif viz_method == 'vtk':

    # Step 1
    # Convert meshes to actors, providing color and opacity
    # options, you can provide vertex colors to color mesh vertices
    # however these will only be relevant for vizmethod=vtk
    post_actor = trimesh_vtk.mesh_actor(motor_neuron_mesh,
                                    opacity=1.0,
                                    color=(1,0,1))
    pre_actor = trimesh_vtk.mesh_actor(premotor_mesh,
                                   opacity=1.0,
                                   color=(0,1,0))
    print("A VTK window should have popped up behind you")
    print("WARNING YOU NEED TO CLOSE IT BY PRESSING Q TO MOVE ON")
    # step 2
    # render them interactively with this function
    # passing a list of actors
    # can optionally specify a path to save a static image
    trimesh_vtk.render_actors([pre_actor, post_actor])

    # vtk controls
    # pinch movements:  to zoom in and out
    # ctrl+wheel: on a mouse to do the same
    # shift+drag: to pan
    # mouse over + f: to zoom to where you point and recenter camera there
    # w: wireframe visualization
    # s: surface visualization
    # q: to exit visualization
    


In [ ]:
# in vtk and vtkplotter there is more camera control
# so this is how you would automatically center the camera
# on the first synapse between these neurons
max_synapses = post_synapse_df[post_synapse_df.pre_pt_root_id == max_input_neuron]

# get the location of the first synapse (change 0 to 1 or 2 to look at others)
syn_pos =convert_to_nm(max_synapses.iloc[[0]].post_pt_position)

# create a camera object pointed at the synapse
camera = trimesh_vtk.oriented_camera(syn_pos, backoff=20)


if viz_method == 'vtk':
    print("A VTK window should have popped up behind you")
    print("WARNING YOU NEED TO CLOSE IT BY PRESSING Q TO MOVE ON")
    # pass the camera to the render_actors function to control camera
    trimesh_vtk.render_actors([post_actor, pre_actor], camera=camera)
if viz_method == 'itkwidgets':
    print("this should have changed the colors and the camera position in the widget above")
    prepostviewer.geometry_colors = np.array([[.8,0,.8], [0,1,0]], dtype=np.float32)
    prepostviewer.camera = camera

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
    <h3> Visualizing synapses </h3>
Now what if we just want to see the synapse locations without the mesh of the other side
</div>

In [ ]:
# synapse sizes tend to distributed widely
# so we are going to color and size them proportional to the log
syn_color = np.log(post_synapse_df['score'].values)
syn_size = 100*np.log(post_synapse_df['score'].values)

# this will produce spheres at each point with sizes in nm
# and color values that are mapped through a colormap
# in VTK only you can pass explicit colors as well
syn_actor = trimesh_vtk.point_cloud_actor(syn_pos_nm, size=syn_size, color=syn_color)

if viz_method == 'vtk':
    print('remember the window pops up behind')
    trimesh_vtk.render_actors([post_actor, syn_actor])
if viz_method == 'itkwidgets':
    syn_pd = syn_actor.GetMapper().GetInput()
    synviewer = view(geometries=[post_poly_data, syn_pd],
                  geometry_colors=['m','g'],
                  ui_collapsed=True)
    display(synviewer)


<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
    <h3> Working with Skeletons </h3>
    
Just as the 3D mesh of the neuron is a reduced data representation of the imagery, one can further reduce a 3D mesh into a skeleton format which can benefit certain types of analyses by making it faster/ more optimal. The <a href = "https://meshparty.readthedocs.io/en/latest/guide/skeletons.html" >"skeletonize"</a> library within meshparty allows us to generate these skeletons from a mesh. Note that this algorithm generates a skeleton where skeleton points generated are a subset of the mesh points. For a few cells however, we have already generated these and provide this in the data directory.
    
The Meshwork library allows us to easily store these representations and helps us relate them to each other. A Meshwork object is a data structure that is designed to have three main components that are kept in sync with mesh and skeleton indices: 
    
    mesh: a standard meshparty mesh
    skeleton: a standard meshparty skeleton,
    anno : is a class that holds dataframes and adds some extra info to keep track of indexing. 
    
In addition, the meshwork object itself has a number of skeleton-like functions that know when to use mesh data and when to use skeleton data. For the most part of this tutorial, we will focus on using the skeleton object.

</div>

In [ ]:
from meshparty import meshwork


<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
Attributes of meshworks objects:
    
    nrn.mesh, nrn.skeleton, nrn.anno

    
Let us now try to visualize the skeleton: nrn.skeleton:
</div>

In [ ]:
test = convert_to_nm(soma_df.pt_position[soma_df.pt_root_id == motor_neuron])[0]
test[1]

In [ ]:
#convert our mesh object into a meshwork object
motor_neuron_mw = meshwork.Meshwork(motor_neuron_mesh, seg_id=motor_neuron,voxel_resolution=[4.3,4.3,45])

#create a skeleton for this neuron
motor_neuron_mw.skeletonize_mesh(soma_pt=soma_point)

# #since we have them, let's add the synapses onto this neuron as an annotation 
motor_neuron_mw.add_annotations('post_synapses', post_synapse_df, point_column='post_pt_position')

In [ ]:
#Let us try to visualize the skeleton:
# Visualize the whole skeleton 
skeleton_actor= trimesh_vtk.skeleton_actor(motor_neuron_mw.skeleton,color=(0.5,0,0.2))
if viz_method=='vtk':
    
    trimesh_vtk.render_actors([skeleton_actor])
elif viz_method=='itkwidgets':
    sk_pd = skeleton_actor.GetMapper().GetInput()
    skviewer = view(geometries=[sk_pd],
                  geometry_colors=['k'],
                  ui_collapsed=True)
    display(skviewer)

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
    <h3>Indexing And Selecting Sets of Interesting Points</h3>

Ok, you are probably really fried by now, but here is one important thing to remember about this meshworks object. The meshworks object contains a mesh with lots of vertices and a skeleton which holds a subset of these vertices. Therefore, in python these points have different "indices" in the mesh and skeleton. For example, if the mesh contains 10000 vertices, the indexing of those would run from 0 - 9999. The skeleton, which contains a subset of 100 of these would have indexing from 0-99. How would you figure out which of the mesh vertices these correspond to?
    
Luckily, we have some really nifty functions that help us distinguish those:
    
Let us first look at some attributes in the meshworks objects:
    
</div>

In [ ]:
#Branch points
print("Branch points with mesh indices: \n", nrn.branch_points)
print("Branch points with skeleton indices: \n",nrn.branch_points_skel)

# nrn.branch_points returns a list of branch points of the cell as a mesh index.
# What it's actually doing is finding the branch points of the skeleton, 
# then looking up a single mesh vertex that corresponds to that skeleton vertex.

In [ ]:
#End points
print("End points with mesh indices: \n", nrn.end_points)
print("End points with skeleton indices: \n",nrn.end_points_skel)



In [ ]:
#Root - point associated with the root node
print("Root point with mesh indices: ", nrn.root)
print("Root point with skeleton indices: ",nrn.root_skel)


<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
    Finally One last nifty function for subselecting points: downstream points.
    For a given point, downstream points are defined as points on paths from endpoints to the root which are further than the given point. For example, if the skeleton path is : A-B-C-D-E where A is the root, D and E are downstream points of C. With branching, this can be more complex. To find the downstream points from say the 9th branch point, we can do:
    
</div>

In [ ]:
#Downstream points

nrn.downstream_of(nrn.branch_points[7])

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
    <h3> Masking</h3>
Just like meshparty meshes, we can mask the meshwork object. Like all basic meshwork functions, the expected input is in mesh vertices. Importantly, doing so will be synchronized across the mesh, the skeleton, and annotations. Do not use the nrn.mesh.apply_mask or nrn.skeleton.apply_mask functions
    
</div>

In [ ]:
nrn.reset_mask() #VERY IMPORTANT - needed to ensure the mask query is run on the whole object

full_object_skeleton = trimesh_vtk.skeleton_actor(nrn.skeleton, color=(0.5,0.5,0.5), line_width=1, opacity=0.5, )

nrn.apply_mask(nrn.downstream_of(nrn.branch_points[7]).to_mesh_mask)

masked_object_skeleton = trimesh_vtk.skeleton_actor(nrn.skeleton, color=(1,0,0), line_width=4, opacity=0.5, )
nrn.reset_mask() # if you are going to do any more operations after this, make sure this is reset
if viz_method=='vtk': 
    trimesh_vtk.render_actors([full_object_skeleton, masked_object_skeleton])
elif viz_method=='itkwidgets':
    fos_pd = full_object_skeleton.GetMapper().GetInput()
    mos_pd = masked_object_skeleton.GetMapper().GetInput()
    viewer = view(geometries=[fos_pd, mos_pd], ui_collapsed=True)
viewer

In [ ]:
# with itkwidgets you have to set colors after setting up rendering
# due to a bug, the widget should change colors when you do this
if viz_method=='itkwidgets':
    viewer.geometry_colors = np.array([[0,0,0], [1,0,0]], dtype=np.float32)

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
    <h3> Annotations</h3>
`nrn.anno` has set of dataframes containing some additional information for analysis. To find out what information it contains, look at the table_names attribute: 
    
</div>

In [ ]:

print(nrn.anno.table_names)
#This shows that there are four dataframes:
#'is_axon' is the annotation of mesh vertices on the axon
#'lvl2_ids' gives the annotation of corresponding level 2 ids
#'post_syn' is the annotation of synapses for which this cell is the postsynaptic cell
#'pre_syn' is the annotation of synapses for which this cell is the presynaptic cell

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
    
How do annotations work?

Let's look at the pre_syn annotation. You can access the data as a dataframe with `.df`
    
</div>

In [ ]:
nrn.anno.pre_syn.df

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
The in-built properties and functions of a meshworks object allow you to connect the dataframe with mesh/skeleton information and operations. 
    
For example, you can get the mesh index for each row with `.mesh_index`. 
</div>

In [ ]:
nrn.anno.pre_syn.mesh_index

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
 
And the skeleton index for each row with `.skel_index`. 
</div>

In [ ]:
nrn.anno.pre_syn.skel_index

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
These mesh indices can be linked up to the meshwork class functions trivially using this information:
    </div>

In [ ]:
nrn.distance_to_root(nrn.anno.pre_syn.mesh_index)/1000 # in microns

#This gives the distance from each point to the root in microns

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

Finally, we will learn how to filter and mask these results for targeted analysis

</div>

In [ ]:
nrn.anno.pre_syn.filter_query( nrn.anno.is_axon.mesh_mask).df